# Project: Create an AI-Powered Chatbot Using AutoGen to retrieve Weather Information of a city 
                                                                                               
We will build a chatbot using the AutoGen framework that takes a city name (assume London) from the user, retrieves real-time weather information, and sends it back as a response to the user.                                                                                         

AutoGen is an open-source programming framework for building AI agents(AI chatbots), therefore, first we will install pyautogen library and then import autogen from it. Since, our autogen chatbot has to fetch weather data from an external API like OpenWeatherMap, therefore, requests module will be needed to fetch the information from the external API

Once we installed all the libraries and dependencies, then we have to create a function to Get Weather Information of a city. To get real time weather data, we can use API of OpenWeatherMap. Visit the link https://openweathermap.org/api to get free API key

After this, we have to create the chatbot class in which we have to call the above function so that when a user asks the question about London's weather, the bot will first process the question and then using function call can retrieve the weather report.

After creating the chatbot class, we have to create the user class. Finally, create the two objects using chatbot class and user class. These objects will call these classes which ultimately build the automatic communication between the user and the chatbot. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/AutoGen_Project')

# Install AutoGen and Required Libraries

In [3]:
!pip install pyautogen

In [4]:
import autogen
import requests   

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


# Set Up a Function to Get Weather Information of London

We have to define a function in such a way that when we pass OpenWeatherMap API key as input, then return statement should give us weather report as output

In [5]:
API_KEY = "41c68af14e3fa28c5831c12347e6f86c"  #OpenWeatherMap API key to fetch weather for London
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"  #Base API URL for fetching weather data

def get_weather():  # function
    params = {"q": "London", "appid": API_KEY, "units": "metric"}  # parameters: q(city),appid(API key), units(degree C)
    response = requests.get(BASE_URL, params=params)  #Sends a get request to OpenWeatherMap API
    if response.status_code == 200:  #Checks if the API request was successful 
        data = response.json()   #Converts the API response (bidefault in JSON format) into a Python dictionary
        weather = data["weather"][0]["description"] #Extracts weather description
        temperature = data["main"]["temp"]  #Extracts temperature (in Celsius)
        return f"Weather in London: {weather}, Temperature: {temperature}°C" #Returns a weather report.
    else:
        return "Sorry, I couldn't fetch the weather for London." #If API  is invalid, return an error message.

# We have to  define a chatbot agent called WeatherBot that responds to messages related to weather in London using the AutoGen framework.



In [6]:
class WeatherAgent(autogen.AssistantAgent): #Creates a chatbot class WeatherAgent, inheriting from AssistantAgent in AutoGen.
    def __init__(self, name="WeatherBot"): #Constructor to initialize the chatbot with the name "WeatherBot".
        super().__init__(name=name) #Calls the parent class (AssistantAgent) constructor to set the chatbot's name.

    def process_message(self, message, sender): #Defines how the chatbot processes user messages.
        if "weather" in message.lower() and "london" in message.lower(): #Checks if the message contains both "weather" and "London".
            return get_weather()  #Calls the get_weather() function to fetch London’s weather.
        return "I can only provide weather updates for London." #Sends a default message if the request is about another city.



# This UserAgent class represents the user interacting with the chatbot in the AutoGen framework. It allows the user to send messages to another class WeatherAgent and receive responses.

In [7]:
class UserAgent(autogen.UserProxyAgent): #Creates a user agent that inherits from UserProxyAgent in AutoGen.
    def __init__(self, name="User"): #Constructor to initialize the user with the name "User"
        super().__init__(name=name)  #Calls the parent class constructor to set the user's name.

    def send_message(self, message, receiver):  #Defines how the user sends messages to another agent
        print(f"\nUser: {message}")  #Prints the user's message 
        response = receiver.process_message(message, self)  #Sends the message to the receiver (e.g., WeatherBot) and gets the response.
        print(f"\n{receiver.name}: {response}\n") #Prints the receiver's (WeatherBot's) response.


# This part of the code runs the chatbot system when the script is executed.

Here, we will call both the classes: UserAgent, WeatherAgent using their objects to build the communication between user and the chatbot

In [8]:
if __name__ == "__main__": #Ensures the code inside only runs when the script is executed, not when imported.
    user = UserAgent() #Create object user using UserAgent class(representing the person interacting with the chatbot).
    bot = WeatherAgent() #Create object bot using  WeatherAgent class(representing the chatbot)

    user.send_message("What is the weather in London?", bot) #User sends a message to the chatbot asking for the weather in London


User: What is the weather in London?

WeatherBot: Weather in London: clear sky, Temperature: 3.78°C



Test the Chatbot on Anaconda Prompt: python autogen_demo.py

Note: If we want the chatbot to fetch weather information for any city, we need to modify the process_message function to extract the city name from the user’s message. Then, we can pass this city name to the get_weather(city) function.